In [608]:
%load_ext autoreload
%autoreload 2

import time
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import uniform, randint
from sklearn.preprocessing import MinMaxScaler
import xarray as xr
#import pymaise 



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [609]:
import psutil
import GPUtil

def get_gpu_utilization():
    # Get the list of available GPUs
    gpus = GPUtil.getGPUs()
    
    if len(gpus) == 0:
        print("No GPU found.")
        return
    
    for gpu in gpus:
        print(f"GPU {gpu.id}:")
        print(f"  - GPU name: {gpu.name}")
        print(f"  - GPU load: {gpu.load * 100:.2f}%")
        print(f"  - GPU memory total: {gpu.memoryTotal} MB")
        print(f"  - GPU memory used: {gpu.memoryUsed} MB")
        print(f"  - GPU memory free: {gpu.memoryFree} MB")
        print(f"  - GPU memory utilization: {gpu.memoryUtil * 100:.2f}%")
        print()

def main():
    # Print CPU utilization
    print("CPU utilization:", psutil.cpu_percent(interval=1))
    
    # Print GPU utilization
    get_gpu_utilization()

if __name__ == "__main__":
    main()


CPU utilization: 4.7
GPU 0:
  - GPU name: NVIDIA GeForce RTX 4090
  - GPU load: 98.00%
  - GPU memory total: 24564.0 MB
  - GPU memory used: 24212.0 MB
  - GPU memory free: 4.0 MB
  - GPU memory utilization: 98.57%

GPU 1:
  - GPU name: NVIDIA GeForce RTX 4090
  - GPU load: 10.00%
  - GPU memory total: 24564.0 MB
  - GPU memory used: 7678.0 MB
  - GPU memory free: 16538.0 MB
  - GPU memory utilization: 31.26%



In [610]:
from pyMAISE.datasets import load_anomaly
from pyMAISE.preprocessing import train_test_split, scale_data, one_hot_encode, SplitSequence
import pyMAISE as mai

In [611]:
%%time

global_settings = mai.init(
    problem_type=mai.ProblemType.CLASSIFICATION,   # Define a regression problem
    #cuda_visible_devices="-1"                  # Use CPU only/ Delete line when run on GPU
    verbosity = 1
)

CPU times: user 50 µs, sys: 5 µs, total: 55 µs
Wall time: 58.7 µs


In [612]:
# call handler
input_path = "/home/jacc/pyMAISE/pyMAISE/datasets/DTL.npy"
output_path = "/home/jacc/pyMAISE/pyMAISE/datasets/DTL_labels.npy"
#print(mai.__file__)
#False, False, False: WORKS       X.shape = (1077, 4500, 14), Y.shape = (1077, 1)  one hot
#False, True, False: WORKS       X.shape = (1077, 4500, 14), Y.shape = (1077, 1)
#False, False, True:  ?      X.shape = (1077, 4500, 14), Y.shape = (1077, 4500, 1)
#False, True, True:   ?      X.shape = (1077, 4500, 14), Y.shape = (1077, 4500, 1)
#True, False, True: WORKS        X.shape = (4846500, 14), Y.shape = (4846500, 1)
#True, True, True: WORKS         X.shape = (4846500, 14), Y.shape = (4846500, 1)
cond1 = True
cond2 = False
cond3 = True
inputs, outputs = load_anomaly([input_path, output_path], cond1, cond2, cond3)


In [613]:
inputs

<xarray.DataArray (timesteps: 4846500, features: 14)>
array([[-6.07100e+00,  5.55120e+00, -1.42220e+00, ..., -4.60370e-02,
         2.17170e+03,  1.27100e+00],
       [-4.85030e+00,  4.33040e+00, -2.64290e+00, ...,  1.50000e-02,
         2.17250e+03, -1.33776e+01],
       [-4.85030e+00,  1.88890e+00, -1.42220e+00, ...,  4.55190e-02,
         2.17320e+03, -8.49500e+00],
       ...,
       [-2.17470e+00, -7.57620e+00, -3.39550e+00, ..., -1.07070e-01,
         2.17220e+03,  1.19730e+01],
       [-4.61620e+00, -3.91400e+00, -5.83700e+00, ..., -1.68110e-01,
         2.17180e+03,  9.53200e+00],
       [-3.39550e+00, -6.35550e+00, -5.83700e+00, ..., -4.60370e-02,
         2.17150e+03,  9.53200e+00]], dtype=float32)
Coordinates:
  * timesteps  (timesteps) int64 0 1 2 3 4 ... 4846496 4846497 4846498 4846499
  * features   (features) <U18 'A+IGBT-I: current' ... 'DV/DT'

In [614]:
outputs

<xarray.DataArray (timesteps: 4846500, features: 1)>
array([['Run'],
       ['Run'],
       ['Run'],
       ...,
       ['Fault'],
       ['Fault'],
       ['Fault']], dtype=object)
Coordinates:
  * timesteps  (timesteps) int64 0 1 2 3 4 ... 4846496 4846497 4846498 4846499
  * features   (features) <U5 'Class'

In [615]:
outputs = one_hot_encode(outputs)
outputs

<xarray.DataArray (timesteps: 4846500, features: 2)>
array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]])
Coordinates:
  * timesteps  (timesteps) int64 0 1 2 3 4 ... 4846496 4846497 4846498 4846499
  * features   (features) object 'Class_Fault' 'Class_Run'

In [616]:
data = xr.concat([inputs, outputs], dim="features")
data

<xarray.DataArray (timesteps: 4846500, features: 16)>
array([[ -6.0710001 ,   5.55119991,  -1.42219996, ...,   1.27100003,
          0.        ,   1.        ],
       [ -4.85029984,   4.33039999,  -2.64289999, ..., -13.37759972,
          0.        ,   1.        ],
       [ -4.85029984,   1.88890004,  -1.42219996, ...,  -8.49499989,
          0.        ,   1.        ],
       ...,
       [ -2.17470002,  -7.57620001,  -3.39549994, ...,  11.97299957,
          1.        ,   0.        ],
       [ -4.61619997,  -3.91400003,  -5.83699989, ...,   9.53199959,
          1.        ,   0.        ],
       [ -3.39549994,  -6.35550022,  -5.83699989, ...,   9.53199959,
          1.        ,   0.        ]])
Coordinates:
  * timesteps  (timesteps) int64 0 1 2 3 4 ... 4846496 4846497 4846498 4846499
  * features   (features) object 'A+IGBT-I: current' ... 'Class_Run'

In [617]:
split_sequence = SplitSequence(
    10, 
    1, 
    0, 
    sequence_inputs=data.coords["features"].values[:-2], 
    sequence_outputs=data.coords["features"].values[-2:],
)
inputs, outputs = split_sequence.split(data)

inputs

<xarray.DataArray (samples: 4846491, timesteps: 10, features: 14)>
array([[[-6.07100010e+00,  5.55119991e+00, -1.42219996e+00, ...,
         -4.60369997e-02,  2.17169995e+03,  1.27100003e+00],
        [-4.85029984e+00,  4.33039999e+00, -2.64289999e+00, ...,
          1.49999997e-02,  2.17250000e+03, -1.33775997e+01],
        [-4.85029984e+00,  1.88890004e+00, -1.42219996e+00, ...,
          4.55190018e-02,  2.17319995e+03, -8.49499989e+00],
        ...,
        [-3.62960005e+00,  1.88890004e+00, -5.08440018e+00, ...,
         -4.60369997e-02,  2.17239990e+03, -1.17100000e+00],
        [-7.29180002e+00,  1.88890004e+00, -3.86360002e+00, ...,
          1.49999997e-02,  2.17230005e+03, -1.33781004e+01],
        [-7.29180002e+00,  3.10969996e+00, -2.64289999e+00, ...,
         -4.60369997e-02,  2.17280005e+03, -1.17100000e+00]],

       [[-4.85029984e+00,  4.33039999e+00, -2.64289999e+00, ...,
          1.49999997e-02,  2.17250000e+03, -1.33775997e+01],
        [-4.85029984e+00,  1.88890004e+00, -1.42219996e+00, ...,
          4.55190018e-02,  2.17319995e+03, -8.49499989e+00],
        [-7.29180002e+00,  1.88890004e+00, -1.42219996e+00, ...,
          4.55190018e-02,  2.17300000e+03, -2.31439991e+01],
...
        [-4.61619997e+00, -8.79699993e+00, -4.61619997e+00, ...,
          1.49999997e-02,  2.17180005e+03,  4.64900017e+00],
        [-2.17470002e+00, -7.57620001e+00, -3.39549994e+00, ...,
         -1.07069999e-01,  2.17219995e+03,  1.19729996e+01],
        [-4.61619997e+00, -3.91400003e+00, -5.83699989e+00, ...,
         -1.68109998e-01,  2.17180005e+03,  9.53199959e+00]],

       [[-7.05770016e+00, -7.57620001e+00, -3.39549994e+00, ...,
          1.06559999e-01,  2.17180005e+03,  7.09000015e+00],
        [-4.61619997e+00, -6.35550022e+00, -7.05770016e+00, ...,
         -4.60369997e-02,  2.17160010e+03,  7.09100008e+00],
        [-3.39549994e+00, -8.79699993e+00, -2.17470002e+00, ...,
         -4.60369997e-02,  2.17180005e+03,  1.19740000e+01],
        ...,
        [-2.17470002e+00, -7.57620001e+00, -3.39549994e+00, ...,
         -1.07069999e-01,  2.17219995e+03,  1.19729996e+01],
        [-4.61619997e+00, -3.91400003e+00, -5.83699989e+00, ...,
         -1.68109998e-01,  2.17180005e+03,  9.53199959e+00],
        [-3.39549994e+00, -6.35550022e+00, -5.83699989e+00, ...,
         -4.60369997e-02,  2.17150000e+03,  9.53199959e+00]]])
Coordinates:
  * samples    (samples) int64 0 1 2 3 4 ... 4846487 4846488 4846489 4846490
  * timesteps  (timesteps) int64 0 1 2 3 4 5 6 7 8 9
  * features   (features) object 'A+IGBT-I: current' ... 'DV/DT'

In [618]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler






In [619]:
x_train, x_test, y_train, y_test = train_test_split(inputs, outputs, test_size=0.3)



In [620]:
x_train_scaled, x_test_scaled, x_scaler = scale_data(x_train, x_test, scaler=MinMaxScaler())



In [621]:
y_train_scaled, y_test_scaled, y_scaler = scale_data(y_train, y_test, scaler=MinMaxScaler())

In [622]:
%%time


x_train_sampled = x_train_scaled#[:2000]  
y_train_sampled = y_train_scaled#[:2000]

x_test_sampled = x_test_scaled#[:800]
y_test_sampled = y_test_scaled#[:800]


CPU times: user 1.17 ms, sys: 4.24 ms, total: 5.41 ms
Wall time: 5.4 ms


<xarray.DataArray (samples: 800, features: 2)>
array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]])
Coordinates:
  * samples   (samples) int64 1344578 1863762 3980206 ... 311599 3628687 2911715
  * features  (features) object 'Class_Fault' 'Class_Run'

In [623]:
x_train_sampled.shape

(2000, 10, 14)

In [624]:
y_train_sampled.shape[-1]

2

In [640]:
lstm_structure = {
    "LSTM_input": {
        "units": 50,
        "return_sequences": True,
        "input_shape": (x_train_scaled.shape[1], x_train_scaled.shape[2]),
    },
    "LSTM": {
        "units": 50,
        "activation": "tanh",
        "recurrent_activation": "sigmoid",
    },
    "Dense": {
        "units": y_train_scaled.shape[1],
        "activation": "softmax",
    },
}

fitting = {
    "batch_size": 64,
    "epochs": 5,
    "validation_split":0.15,
    "callbacks": [
        ReduceLROnPlateau(
            monitor='val_mean_absolute_error',
            factor=0.8,
            patience=2,
            min_lr=0,
            verbose=1,
        ),
        EarlyStopping(
            monitor="val_mean_absolute_error",
            patience=3,
        )
    ]
}
adam = {
    "learning_rate": mai.Float(min_value=0.00001, max_value=0.001),
    "clipnorm": mai.Float(min_value=0.8, max_value=1.2),
    "clipvalue": mai.Float(min_value=0.3, max_value=0.7),
}
compiling = {
    "loss": "mean_absolute_error",
    "metrics": ["mean_absolute_error"],
}

model_settings = {
    "models": ["LSTM"],
    "LSTM": {
        "structural_params": lstm_structure,
        "optimizer": "Adam",
        "Adam": adam,
        "compile_params": compiling,
        "fitting_params": fitting,
    },
}
tuner = mai.Tuner(x_train_sampled, y_train_sampled, model_settings=model_settings)

In [641]:
x_train_sampled.shape

(2000, 10, 14)

In [642]:
lstm_structure

{'LSTM_input': {'units': 50,
  'return_sequences': True,
  'input_shape': (10, 14)},
 'LSTM': {'units': 50,
  'activation': 'tanh',
  'recurrent_activation': 'sigmoid'},
 'Dense': {'units': 2, 'activation': 'softmax'}}

In [643]:
y_train_sampled.shape

(2000, 2)

In [644]:

start = time.time()

bayesian_search_configs = tuner.nn_bayesian_search(
    objective="accuracy_score",
    max_trials=10,
    cv=2,
)

print("Hyperparameter tuning took " + str((time.time() - start) / 60) + " minutes to process.")

Trial 10 Complete [00h 00m 07s]
accuracy_score: 0.8414999999999999

Best accuracy_score So Far: 0.8414999999999999
Total elapsed time: 00h 01m 09s

Top Configurations

-- LSTM | Training Time: 00:01:09
Adam_learning_rate: 0.0009456788011137315
Adam_clipnorm: 1.130047321405258
Adam_clipvalue: 0.6217470635658112
Hyperparameter tuning took 1.162446343898773 minutes to process.


In [630]:
x_train_scaled

<xarray.DataArray (samples: 3392543, timesteps: 10, features: 14)>
array([[[0.59629781, 0.33542024, 0.79247002, ..., 0.51306262,
         0.9330013 , 0.49838974],
        [0.59324178, 0.34559398, 0.79247002, ..., 0.51214572,
         0.93313695, 0.52990141],
        [0.59171365, 0.35356079, 0.79288106, ..., 0.51397968,
         0.93268487, 0.55650795],
        ...,
        [0.65344732, 0.58793759, 0.79534662, ..., 0.50572665,
         0.93336299, 0.63028292],
        [0.62135815, 0.6060891 , 0.79781252, ..., 0.50160005,
         0.93336299, 0.62418868],
        [0.57765546, 0.60672608, 0.7961687 , ..., 0.49884904,
         0.93291091, 0.62181044]],

       [[0.59996515, 0.23446547, 0.62110339, ..., 0.49884904,
         0.9399182 , 0.37021256],
        [0.60027075, 0.24432615, 0.64822658, ..., 0.49426393,
         0.94000859, 0.36535698],
        [0.60027075, 0.25262167, 0.68767759, ..., 0.49059601,
         0.94014424, 0.38468018],
...
        [0.21152113, 0.60831832, 0.80438769, ..., 0.50297563,
         0.93539737, 0.47302185],
        [0.22894552, 0.61086591, 0.810141  , ..., 0.50205857,
         0.93566855, 0.4560769 ],
        [0.24026135, 0.61182135, 0.82575712, ..., 0.50205857,
         0.93575895, 0.44155973]],

       [[0.59935384, 0.60736298, 0.801922  , ..., 0.49701494,
         0.92680779, 0.52623   ],
        [0.59935384, 0.60672608, 0.80151103, ..., 0.49701494,
         0.92671734, 0.52623495],
        [0.59996515, 0.60640762, 0.80151103, ..., 0.49701494,
         0.92635565, 0.52381708],
        ...,
        [0.60027075, 0.60704452, 0.80110009, ..., 0.49701494,
         0.92671734, 0.51655354],
        [0.60088198, 0.6060891 , 0.80027818, ..., 0.49884904,
         0.92658169, 0.5153446 ],
        [0.60118761, 0.60545233, 0.80151103, ..., 0.49884904,
         0.92725987, 0.51171779]]])
Coordinates:
  * samples    (samples) int64 2198761 766455 2678728 ... 3845532 3784053
  * timesteps  (timesteps) int64 0 1 2 3 4 5 6 7 8 9
  * features   (features) object 'A+IGBT-I: current' ... 'DV/DT'

In [631]:
y_train_scaled.shape

(3392543, 2)

KeyboardInterrupt: 

In [ ]:
x_train.size


In [ ]:
plt.plot(quick_history.history['loss'], label='Training loss')
plt.plot(quick_history.history['val_loss'], label='Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Training vs Validation Loss')
plt.show()

In [ ]:
plt.plot(quick_history.history['accuracy'], label='Training accuracy')
plt.plot(quick_history.history['val_accuracy'], label='Validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Training vs Validation Accuracy')
plt.show()

In [ ]:
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay


y_test_sampled_np = y_test_sampled.values 

y_true_classes = np.argmax(y_test_sampled_np, axis=1)

y_pred_prob = model.predict(x_test_sampled.values) 

y_pred_classes = np.argmax(y_pred_prob, axis=1)


cm = confusion_matrix(y_true_classes, y_pred_classes)


disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap=plt.cm.Blues)
plt.show()

In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

fpr, tpr, _ = roc_curve(y_true_classes, y_pred_prob[:, 1])
roc_auc = auc(fpr, tpr)

# Plot the ROC curve
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = {:.2f})'.format(roc_auc))
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()